In [1]:
import os
import pandas as pd
import numpy as np

from loguru import logger
from tracks_import import read_from_csv

from config_file import dataset_dir_raw, recording, n

In [2]:
if not os.path.isdir(dataset_dir_raw):
    raise Exception("Directory does not exist")


# Create paths to csv files
tracks_file = dataset_dir_raw + recording + "_tracks.csv"
tracks_meta_file = dataset_dir_raw + recording + "_tracksMeta.csv"
recording_meta_file = dataset_dir_raw + recording + "_recordingMeta.csv"

if not os.path.isfile(tracks_file) or not os.path.isfile(tracks_meta_file) or not os.path.isfile(recording_meta_file):
    raise Exception("File does not exist")
# Load csv files
logger.info("Loading csv files {}, {} and {}", tracks_file, tracks_meta_file, recording_meta_file)
tracks, static_info, meta_info = read_from_csv(tracks_file, tracks_meta_file, recording_meta_file,
                                                   include_px_coordinates=True)
df_tracks = pd.DataFrame(tracks)
df_static_info = pd.DataFrame(static_info)
df_meta_info = pd.DataFrame(meta_info.items())
display(df_tracks)

# Select the track(s)
track_ids = list(range(n))  

selected_tracks = df_tracks[df_tracks['trackId'].isin(track_ids)]

# Change the position to the distance by subtracting the starting position from the position lists
selected_tracks.loc[:, 'xCenter'] = selected_tracks['xCenter'].apply(lambda num: [x - num[0] for x in num[:]])
selected_tracks.loc[:, 'yCenter'] = selected_tracks['yCenter'].apply(lambda num: [x - num[0] for x in num[:]])

# Select distance, velocity, and acceleration columns
columns_to_select = ['xCenter', 'yCenter', 'xVelocity', 'yVelocity', 'xAcceleration', 'yAcceleration']
selected_data = selected_tracks[columns_to_select]

# Unstack the selected_data DataFrame to flatten the lists in the 'xCenter', 'yCenter', etc. columns
selected_data_unstacked = selected_data.apply(lambda x: np.concatenate(x.values))

# Create a new DataFrame
new_dataframe = pd.DataFrame(selected_data_unstacked)
display(new_dataframe)

output_directory = '../../data/processed/'
file_name = f'x_y_recording_{recording}_range_{n}.csv'
output_file_path = os.path.join(output_directory, file_name)

# Save the new_dataframe as a CSV file in the data/processed directory
new_dataframe.to_csv(output_file_path, index=False)


2023-12-30 15:18:38.287 | INFO     | __main__:<module>:13 - Loading csv files ../../data/raw/inD/data/00_tracks.csv, ../../data/raw/inD/data/00_tracksMeta.csv and ../../data/raw/inD/data/00_recordingMeta.csv


,recordingId,trackId,frame,trackLifetime,xCenter,yCenter,heading,width,length,xVelocity,...,latAcceleration,center,bbox,xCenterVis,yCenterVis,centerVis,widthVis,lengthVis,headingVis,bboxVis
0,0,0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[131.38519, 131.3383, 131.28821, 131.23861, 13...","[-34.7838, -35.13993, -35.50286, -35.86783, -3...","[261.84968, 261.95002, 262.00456, 262.03306, 2...","[1.76028, 1.76028, 1.76028, 1.76028, 1.76028, ...","[4.03141, 4.03141, 4.03141, 4.03141, 4.03141, ...","[-1.20819, -1.20844, -1.20806, -1.20726, -1.20...",...,"[0.08452, 0.09037, 0.09908, 0.11096, 0.12583, ...","[[131.38519, -34.7838], [131.3383, -35.13993],...","[[[131.97067233266367, -36.90392346450948], [1...","[10345.343282535594, 10341.651137732073, 10337...","[2738.895850217681, 2766.937725433673, 2795.51...","[[10345.343282535594, 2738.895850217681], [103...","[138.60543089660072, 138.60543089660072, 138.6...","[317.435476271312, 317.435476271312, 317.43547...","[98.15032000000002, 98.04998, 97.9954399999999...","[[[10254.239037769681, 2886.1853724283237], [1..."
1,0,1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[135.03907, 135.0677, 135.09607, 135.12432, 13...","[-45.99449, -46.1289, -46.26314, -46.39582, -4...","[275.68596, 275.99277, 276.32133, 276.67232, 2...","[1.84093, 1.84093, 1.84093, 1.84093, 1.84093, ...","[3.92792, 3.92792, 3.92792, 3.92792, 3.92792, ...","[0.72075, 0.72438, 0.72821, 0.73225, 0.73619, ...",...,"[-0.19502, -0.20565, -0.21602, -0.22585, -0.23...","[[135.03907, -45.99449], [135.0677, -46.1289],...","[[[136.149587332072, -47.85759116552825], [135...","[10633.05183563196, 10635.306177831251, 10637....","[3621.631845683296, 3632.215364195585, 3642.78...","[[10633.05183563196, 3621.631845683296], [1063...","[144.95585696620944, 144.95585696620944, 144.9...","[309.2866158380348, 309.2866158380348, 309.286...","[84.31403999999998, 84.00722999999999, 83.6786...","[[[10576.251934510436, 3782.6951066521556], [1..."
2,0,2,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[113.34668, 113.05249, 112.75417, 112.45299, 1...","[-65.72286, -65.90637, -66.09502, -66.28951, -...","[213.33247, 213.3922, 213.45667, 213.53202, 21...","[1.84183, 1.84183, 1.84183, 1.84183, 1.84183, ...","[4.26398, 4.26398, 4.26398, 4.26398, 4.26398, ...","[-7.36258, -7.43408, -7.50676, -7.58034, -7.65...",...,"[-0.63784, -0.64564, -0.64967, -0.64971, -0.64...","[[113.34668, -65.72286], [113.05249, -65.90637...","[[[112.07145015521381, -67.66380168904982], [1...","[8924.980924681935, 8901.816239679849, 8878.32...","[5175.054724280775, 5189.5044042315985, 5204.3...","[[8924.980924681935, 5175.054724280775], [8901...","[145.0267234691561, 145.0267234691561, 145.026...","[335.74816803831635, 335.74816803831635, 335.7...","[146.66753, 146.6078, 146.54333, 146.46798, 14...","[[[8744.876953231256, 5206.7162969361925], [90..."
3,0,3,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[89.96716, 89.92104, 89.87504, 89.83031, 89.78...","[-73.02951, -73.04865, -73.06778, -73.08882, -...","[207.02469, 207.02151, 207.02047, 207.02288, 2...","[3.06624, 3.06624, 3.06624, 3.06624, 3.06624, ...","[19.08486, 19.08486, 19.08486, 19.08486, 19.08...","[-1.16402, -1.13548, -1.1056, -1.07437, -1.041...",...,"[-0.12784, -0.13041, -0.13217, -0.13303, -0.13...","[[89.96716, -73.02951], [89.92104, -73.04865],...","[[[82.16327072838743, -78.73106588201239], [99...","[7084.0644547136935, 7080.432940140471, 7076.8...","[5750.384428453207, 5751.891522749205, 5753.39...","[[7084.0644547136935, 5750.384428453207], [708...","[241.4374511057292, 241.4374511057292, 241.437...","[1502.7525415850314, 1502.7525415850314, 1502....","[152.97531, 152.97849, 152.97953, 152.97712, 1...","[[[6359.878852612863, 5984.2530341253505], [76..."
4,0,4,"[0, 1, 2, 3, 4, 5, 

,xCenter,yCenter,xVelocity,yVelocity,xAcceleration,yAcceleration
0,0.00000,0.00000,-1.20819,-9.06651,-0.01047,0.52307
1,-0.04689,-0.35613,-1.20844,-9.06017,-0.01211,0.55970
2,-0.09698,-0.71906,-1.20806,-9.05198,-0.01427,0.61071
3,-0.14658,-1.08403,-1.20726,-9.04039,-0.01754,0.67524
4,-0.19546,-1.44961,-1.20636,-9.02442,-0.02230,0.75172
...,...,...,...,...,...,...
205270,16.36797,9.65808,13.73136,7.97613,0.08649,-0.19391
205271,16.91658,9.97623,13.73430,7.96779,0.08599,-0.19445
205272,17.46545,10.29410,13.73744,7.95971,0.08581,-0.19465
205273,18.01444,10.61179,13.74080,7.95183,0.08579,-0.19468
